## Notebook Parameters

---

### Notebook Settings

In [ ]:
"""Google Colab settings"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
"""Jupyter settings"""
# Enable autoreload
%load_ext autoreload
%autoreload 2

### Imported Packages

In [ ]:
"""Packages for manipulation of vectors, arrays, dataframes"""
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None) # Change display settings of class

"""Packages for cleaning dataset"""
import json
import string
import unicodedata

"""Packages for data visualization"""
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn

## 1. Preprocessing

---

### 1. 1. Clean OCR Dataset



#### 1. 1. 1. Import the Data

In [ ]:
# Load the dataset
file = '/content/drive/MyDrive/OFF_Drive/raw_data/ocr_fr_text_only.jsonl'

def load_json_data(file):
    """Load the dataset"""
    data = []
    with open(file) as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

ocr_df_base = load_json_data(file)

In [ ]:
# Deep copy of the dataframe to avoid to reload it
ocr_df = ocr_df_base.copy()

In [ ]:
# Brief look at the dataset

print(f"""
    Shape of the dataset: {ocr_df.shape}
    """)
print("""
    head of ocr_df:
    """)
display(ocr_df.head())
print("""
    Content of the first element:
    """)
display(ocr_df.fr_text[0])

#### 1. 1. 2. Format the Dataset

In [ ]:
# Format the content of the column SOURCE
# CREATE BARCODE

def to_barcode(text):
    """Transform strings in barcodes"""
    return int(''.join(text.split('/')[0:-1]))


# Apply function to create barcodes
ocr_df['barcode'] = ocr_df.source.apply(lambda text: to_barcode(text))

In [ ]:
# Format the content of the column TEXT
# CLEAN OCR TEXT

def lower_case(text):
    """Lower case"""
    text = text.lower()
    return text

def replace_slashn(text):
    """Replace '/n' """
    text = text.replace('\n',' ')
    return text

def remove_punc(text):
    """Remove punctuation"""
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def remove_digit(text):
    """Remove digits"""
    return ''.join(word for word in text if not word.isdigit())

def remove_nonalpha(text):
    """Remove nonalpha characters"""
    return ''.join(word for word in text if word.isalpha() or word == ' ')

def remove_accents(text):
    """Remove accents"""
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text) 
        if unicodedata.category(c) != 'Mn'
        )

def remove_specialchar(text):
    """Remove special characters"""
    return ''.join(word for word in text if word.isalpha() or word == ' ')

def text_formatting_workflow():
    """Workflow of text formatting"""
    ocr_df['clean_text'] = ocr_df.fr_text.apply(lambda text: lower_case(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: replace_slashn(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_punc(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_digit(text))
    # ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_nonalpha(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_accents(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_specialchar(text))
    
    return ocr_df


# Apply workflow of text formatting
text_formatting_workflow()

,source,fr_text,barcode,clean_text
0,/319/966/047/6748/1.json,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n",3199660476748,ne eleve abattu en bretagne les eleveurs de bretagne decoupe de podlet noir ferfier labe caracteristiques certifiees fermiereleve en plein air duree delevage jours minimum de cereales alimente avec de vegetaux mineraux et vitamines do mation voir etiquette poidsprix a conserver entre oc et c a consommer cuit a caeurdate limite de co le rheu certifie par certisimmeuble le millepertuis les landes dap produit frais classe a pour toute reclamation sadresser a fermiers dargoat bp ploufragan decoupe et conditionne par ldc bretagne bp quintin homologation n la ce abattoir agree nfr ce fltplt ferlr sat elbretpf origine france yo volaise prixtrg poids net prix a payer kg lot r consommer jusqu au r conserver entre o c et c expedie le loc bretagne lanfains e a au trit aste wwwconsignesdetrfr
1,/319/966/021/9208/1.json,"..\n1cg% LOCAL\nKer\nchAnt\nPOULET\nDecoupes de\nConditionné par D\nBretagne\nOrigine FRANCE. Produit Frais, Classe A\nconsommer cuit à coeur\nNE ELEVÉ PREPARE\n100% LOCAL\nDANS NOTRE REGION\nFILETS POOLET\n720GR\nVOLAILLE\nORIGINE France\nPrixtkg\nPoids net\nPrix a pave\n0,720kg\nO058067878\nA consommer jusqu au\n04/10/18\nCot\nA conserver entre O C et 4C\nExpedie Le\nLOc BRETAGNE 22 LANFAINS\n22.099.002\nCE\n250498-0/181\n01\n3 11199660121\nCONSGNE POUANT REOCLMENT\nwww.cONSIGNT\n",3199660219208,cg local ker chant poulet decoupes de conditionne par d bretagne origine france produit frais classe a consommer cuit a coeur ne eleve prepare local dans notre region filets poolet gr volaille origine france prixtkg poids net prix a pave kg o a consommer jusqu au cot a conserver entre o c et c expedie le loc bretagne lanfains ce consgne pouant reoclment wwwconsignt
2,/319/966/021/9208/2.json,FILETS POULET\n720GR\nORIGINE France\nPrixtkg\nVOLAILLE\nFRANÇAISE\nPoids net\nPrix a payer\n0720kg\nLot\n0058067878\nA consommer jusqu au\n04/10/18\nA conserver entre O C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\nFR\nCE\nBARQUETTE ET\n22.099.002\n250498-0/18 1C\ne01\nPENSEZ\nAU TRI!\nCONSIGNE POUVANTVARIER LOCALEMENT\nFILM PLASTIQUE\nA JETER\n3 |199 66012 19208\nwww.CONSIGNESDETRI.FR\n,3199660219208,filets poulet gr origine france prixtkg volaille francaise poids net prix a payer kg lot a consommer jusqu au a conserver entre o c et c expedie le loc bretagne lanfains fr ce barquette et c e pensez au tri consigne pouvantvarier localement film plastique a jeter wwwconsignesdetrifr
3,/319/966/021/9192/1.json,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 et +4""C\nOFFRE\nSPECIALE\nNE ELEVE PREPARE\n100% LOCAL\nDANS NOTRE REGION\n1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANÇAISE\nPrix/kg\nPaids net\nPrin apayer\n1,000kg\nto\n005806444\nA consommer jusqu au\n27/09/18\nR conserver entre D C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\n2.09.02\n256-106-0/1818\nAUTR

In [ ]:
# Drop superfluous columns

# Rough variant (keeping the raw text as well)
ocr_rough_df = ocr_df[['barcode','clean_text', 'fr_text']].copy()
# Replace column names of the dataset
ocr_rough_df.rename(columns={'fr_text':'raw_text'}, inplace=True)

# Clean variant
ocr_clean_df = ocr_df[['barcode','clean_text']].copy()

In [ ]:
ocr_rough_df.dtypes , ocr_clean_df.dtypes

(barcode        int64
 clean_text    object
 raw_text      object
 dtype: object, barcode        int64
 clean_text    object
 dtype: object)

In [ ]:
"""Uncomment method if needed"""

# Save formatted dataset to csv
# ocr_clean_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/ocr_clean.csv', index = False)
# ocr_rough_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/ocr_clean_vs_rough.csv', index = False)

'Uncomment method if needed'

### 1. 2. Clean OFF Dataset

#### 1. 2. 1. Import the Data

In [ ]:
# Load the dataset
file = '/content/drive/My Drive/OFF_Drive/raw_data/en.openfoodfacts.org.products.csv'
off_df_base = pd.read_csv(file, sep='\t')

In [ ]:
# Deep copy of the dataframe to avoid to reload it
data_off = off_df_base.copy()

In [ ]:
# Brief look at the dataset
print(f"""
    Shape of the dataset: {data_off.shape}
    """)
print("""
    head of data_off:
    """)
display(data_off.head())

#### 1. 2. 2. Format the Dataset

In [ ]:
# Drop columns
data_off_small = data_off[['code', 'pnns_groups_1', 'pnns_groups_2']].copy()

In [ ]:
# Replace empty values with NaN
data_off_small['pnns_groups_1'] = data_off_small['pnns_groups_1'].replace('unknown',np.nan)
data_off_small['pnns_groups_2'] = data_off_small['pnns_groups_2'].replace('unknown',np.nan)

In [ ]:
# Drop rows with NaN
off_df = data_off_small.dropna(subset=['pnns_groups_1', 'pnns_groups_2']).copy()

In [ ]:
# Brief look at the dataset
print(f"""
    Shape of the dataset: {off_df.shape}
    """)
print("""
    head of the dataset:
    """)
display(off_df.head())

In [ ]:
# Replace column names of the dataset
off_df.rename(columns={'code':'barcode'}, inplace=True)

In [ ]:
# Verify columns of the dataset
off_df.dtypes

barcode          object
pnns_groups_1    object
pnns_groups_2    object
dtype: object

In [ ]:
"""Uncomment method if needed"""

# Save formatted dataset to csv
# off_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/off_categories_only.csv', index = False)

### 1. 3. Merge OCR & OFF Datasets

Columns of the final dataset (rough variant):
- `barcode`
- `clean_text`
- `raw_text`
- `pnns_group_1`
- `pnns_group_2`

In [ ]:
# Merge the 2 dataframes
merged_df = ocr_rough_df.merge(off_df, how='left', on='barcode')

In [ ]:
# Brief look at the dataset
print(f"""
    Shape of the dataset: {merged_df.shape}
    """)
print("""
    head of the dataset:
    """)
display(merged_df.head())

In [ ]:
# Drop rows with NaN from the DataFrame
labeled_df = merged_df.dropna(subset=['pnns_groups_1', 'pnns_groups_2'])

In [ ]:
# Brief look at the dataset
print(f"""
    Shape of the dataset: {labeled_df.shape}
    """)
print("""
    head of the dataset:
    """)
display(labeled_df.head())

In [ ]:
# Check the number of barcodes
print(f"""
    Count of unique barcodes: {labeled_df.barcode.nunique()}
    """)


    Count of unique barcodes: 127888
    


In [ ]:
"""Uncomment method if needed"""
# Save sample dataset to csv
# labeled_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/ocr_labeled_rough.csv', index = False)

In [ ]:
# Create a subset dataframe to test the code without waiting time
labeled_df_1K = labeled_df[:1000].copy()

"""Uncomment method if needed"""
# Save sample dataset to csv
# labeled_df_1K.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/ocr_labeled_1K.csv', index = False)

'Uncomment method if needed'